In [1]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda.compiler import SourceModule

In [2]:
###  Double buffering

In [3]:
import unittest
import numpy.testing as npt
class TestGameOfLife(unittest.TestCase):
    def setUp(self):
        self.shape = (6,6)
        self.uut = GameOfLife(self.shape)
        
    def testUpdate(self):
        a = np.array([
                [0,0,0,0,0,0],
                [0,0,0,0,0,0],
                [0,0,1,1,1,0],
                [0,1,1,1,0,0],
                [0,0,0,0,0,0],
                [0,0,0,0,0,0]
            ]).astype(np.bool_)
        self.uut.array = a
        b = np.array([
                [0,0,0,0,0,0],
                [0,0,0,1,0,0],
                [0,1,0,0,1,0],
                [0,1,0,0,1,0],
                [0,0,1,0,0,0],
                [0,0,0,0,0,0]
            ]).astype(np.bool_)
        
        self.uut.run()
        npt.assert_equal(self.uut.array,b)
        self.uut.run()
        npt.assert_equal(self.uut.array,a)

In [4]:
import sys
def runGOFTest():
    suite = unittest.TestLoader().loadTestsFromTestCase(TestGameOfLife)
    unittest.TextTestRunner().run(suite)

In [63]:
class GameOfLife(object):
    def __init__(self,shape):
        self.shape = shape
        self.reset()
        self.init_cuda()
        
    def reset(self):
        self.array = np.random.randint(0,2,(self.shape)).astype(np.bool_)

    def init_cuda(self):
        mod = SourceModule("""
            __global__ void bit_gol3(bool *dest, bool *a,
                unsigned int nbThreadX,unsigned int nbThreadY,unsigned int nbIt)
            {
            
                unsigned int x =  blockIdx.x*blockDim.x + threadIdx.x;
                unsigned int y =  blockIdx.y*blockDim.y + threadIdx.y;
                unsigned int nbRow = blockDim.y*nbThreadY;
                unsigned int nbCol = blockDim.x*nbThreadX;
                unsigned int thid = x + y*nbCol; //thread id
                unsigned int N;
                unsigned int sizeTot = nbRow * nbCol;
                
                
                
                int pout = 0, pin = 1;         //double buffer indices
                unsigned int offIn,offOut;     //buffer offset
                
                extern __shared__ bool tmp[]; // allocated on invocation
                
                tmp[pout * sizeTot + thid] = a[thid];//Put the input in the buffer
                __syncthreads(); 
                
                for(unsigned int i = 0 ; i < nbIt ; ++i ){
                
                    pout = 1 - pout; //swap double buffer indices
                    pin = 1 - pout;
                    
                    offIn = pin * sizeTot; //compute les offset for the indices
                    offOut = pout * sizeTot;
                    
                    if(x > 0 && y > 0 && x < nbCol-1 && y < nbRow-1){
                        N = ( 
tmp[offIn + x-1 + (y-1)*nbCol] + tmp[offIn + x + (y-1)*nbCol] + tmp[offIn + x+1 + (y-1)*nbCol] +
tmp[offIn + x-1 + ( y )*nbCol] +                              + tmp[offIn + x+1 + ( y )*nbCol] +
tmp[offIn + x-1 + (y+1)*nbCol] + tmp[offIn + x + (y+1)*nbCol] + tmp[offIn + x+1 + (y+1)*nbCol] 
                        );
                    }else{
                        N = 0;
                    }
                    
                    tmp[offOut + thid] = ( tmp[offIn + thid] & ((N==2) | (N==3))) 
                                       | (!tmp[offIn + thid] &   N==3); 
                    
                    __syncthreads();   
                }
                dest[thid] = tmp[pout * sizeTot + thid];//Put the buffer in the output
            }
            """)

        self.cuda_func = mod.get_function("bit_gol3")

        self.blockSizeX = 32
        self.blockSizeY = 32
        self.nBlocksX = self.shape[1]//self.blockSizeX + (0 if self.shape[1]%self.blockSizeX == 0 else 1);
        self.nBlocksY = self.shape[0]//self.blockSizeY + (0 if self.shape[0]%self.blockSizeY == 0 else 1);

        

        #To optimize we are going to fill the grid and extract the data that we want.
        #We can then make the border that we want.

        self.nbThreadX = np.uint32(self.nBlocksX)
        self.nbThreadY = np.uint32(self.nBlocksY)
        
        #should be in byte
        self.sharedMemorySize = 2*(self.blockSizeY*self.nBlocksY *
                                  self.blockSizeX*self.nBlocksX )
        print("self.sharedMemorySize",self.sharedMemorySize)
    
    def iterate(self,array,nbIt=1):
        Xtmp = np.zeros((self.blockSizeY*self.nBlocksY,self.blockSizeX*self.nBlocksX)).astype(array.dtype)
        Xtmp[0:self.shape[0],0:self.shape[1]] = array
        Ytmp = np.zeros_like(Xtmp)
       
        self.cuda_func(
                drv.Out(Ytmp), drv.In(Xtmp),self.nbThreadX,self.nbThreadY,np.uint32(nbIt),
                block=(self.blockSizeX,self.blockSizeY,1), grid=(self.nBlocksX,self.nBlocksY),
                shared=self.sharedMemorySize
                )
        Y = Ytmp[0:self.shape[0],0:self.shape[1]]

        return Y



    def run(self,nbIteration=1):
        self.array = self.iterate(self.array,nbIt=nbIteration)

In [90]:
gof = GameOfLife((130,120))

self.sharedMemorySize 40960


In [91]:
gof.run()

In [92]:
runGOFTest()

.

self.sharedMemorySize 2048



----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


In [95]:
gof = GameOfLife((130,120))
%timeit uut.run(1000)

self.sharedMemorySize 40960
100 loops, best of 3: 11 ms per loop


In [94]:
gof = GameOfLife((130,120))

def test():
    for i in range(1000):
        uut.run(1)

%timeit test()

self.sharedMemorySize 40960
1 loops, best of 3: 261 ms per loop


The speed is much better. But we are limited by the shared memory size which is MAX_SHARED_MEMORY_PER_BLOCK:49152 for me. Conclusion it is very fast but limited to small array!

In [30]:
(free,total)=drv.mem_get_info()
print("Global memory occupancy:%f%% free"%(free*100/total))

for devicenum in range(drv.Device.count()):
    device=drv.Device(devicenum)
    attrs=device.get_attributes()

#Beyond this point is just pretty printing
print("\n===Attributes for device %d"%devicenum)
for (key,value) in attrs.items():
    print("%s:%s"%(str(key),str(value)))

Global memory occupancy:36.336357% free

===Attributes for device 0
MAX_THREADS_PER_BLOCK:1024
MAX_BLOCK_DIM_X:1024
MAX_BLOCK_DIM_Y:1024
MAX_BLOCK_DIM_Z:64
MAX_GRID_DIM_X:65535
MAX_GRID_DIM_Y:65535
MAX_GRID_DIM_Z:65535
MAX_SHARED_MEMORY_PER_BLOCK:49152
TOTAL_CONSTANT_MEMORY:65536
WARP_SIZE:32
MAX_PITCH:2147483647
MAX_REGISTERS_PER_BLOCK:32768
CLOCK_RATE:1280000
TEXTURE_ALIGNMENT:512
GPU_OVERLAP:1
MULTIPROCESSOR_COUNT:2
KERNEL_EXEC_TIMEOUT:1
INTEGRATED:0
CAN_MAP_HOST_MEMORY:1
COMPUTE_MODE:DEFAULT
MAXIMUM_TEXTURE1D_WIDTH:65536
MAXIMUM_TEXTURE2D_WIDTH:65536
MAXIMUM_TEXTURE2D_HEIGHT:65535
MAXIMUM_TEXTURE3D_WIDTH:2048
MAXIMUM_TEXTURE3D_HEIGHT:2048
MAXIMUM_TEXTURE3D_DEPTH:2048
MAXIMUM_TEXTURE2D_ARRAY_WIDTH:16384
MAXIMUM_TEXTURE2D_ARRAY_HEIGHT:16384
MAXIMUM_TEXTURE2D_ARRAY_NUMSLICES:2048
SURFACE_ALIGNMENT:512
CONCURRENT_KERNELS:1
ECC_ENABLED:0
PCI_BUS_ID:15
PCI_DEVICE_ID:0
TCC_DRIVER:0
MEMORY_CLOCK_RATE:800000
GLOBAL_MEMORY_BUS_WIDTH:128
L2_CACHE_SIZE:131072
MAX_THREADS_PER_MULTIPROCESSOR:153